In [2]:
import csv
import pandas as pd
import numpy as np

import folium
import folium.plugins as plugins
from folium.plugins import HeatMapWithTime
import ipywidgets as widgets
from IPython.display import HTML
from IPython.display import display

#create the UI for the map and radius selection

#output widget for the map
out = widgets.Output()

#selection widgets for army group and num
radius_sel = widgets.IntSlider(
                    value=25,
                    min=5,
                    max=100,
                    step=5,
                    description='Radius:',
                    disabled=False,
                    continuous_update=False,
                    orientation='horizontal',
                    readout=True,
                    readout_format='d'
                )

#parse and load csv file into a Pandas DataFrame
filename = 'Division.csv'
csvfile = open(filename,'r')
df = pd.read_csv(csvfile,parse_dates=['MAP_DATE'])

#specify the three dates we are interested in
beg = '1941-07-11'
mid = '1941-08-02'
end = '1941-09-10'

#filter the DataFrame for each of these dates and keep only the location and weight columns
beg_data = df.loc[df['MAP_DATE'] == beg,['POINT_Y','POINT_X','Weight_D']]
mid_data = df.loc[df['MAP_DATE'] == mid,['POINT_Y','POINT_X','Weight_D']]
end_data = df.loc[df['MAP_DATE'] == end,['POINT_Y','POINT_X','Weight_D']]

#drop rows with NaN Weight_D values
beg_data.dropna(inplace=True)
mid_data.dropna(inplace=True)
end_data.dropna(inplace=True)

#make sure to interpret the location as a float number
beg_data['POINT_X'] = beg_data['POINT_X'].astype(float)
beg_data['POINT_Y'] = beg_data['POINT_Y'].astype(float)
beg_data['Weight_D'] = beg_data['Weight_D'].astype(float)
mid_data['POINT_X'] = mid_data['POINT_X'].astype(float)
mid_data['POINT_Y'] = mid_data['POINT_Y'].astype(float)
mid_data['Weight_D'] = mid_data['Weight_D'].astype(float)
end_data['POINT_X'] = end_data['POINT_X'].astype(float)
end_data['POINT_Y'] = end_data['POINT_Y'].astype(float)
end_data['Weight_D'] = end_data['Weight_D'].astype(float)

#create the list of list of lists of locations for the heat map animation
#also include weight to incorporate that into the heatmap
beg_heat_data = beg_data.values.tolist()
mid_heat_data = mid_data.values.tolist()
end_heat_data = end_data.values.tolist()

#heat map data to use for every map
heat_data = [beg_heat_data,mid_heat_data,end_heat_data]

#event handler for radius change
def on_radius_change(change):
    new_radius = change['new']
    display_map(new_radius,out)

#assign event handler for radius change
radius_sel.observe(on_radius_change, names='value')

#function to display map for given radius and output widget
def display_map(req_radius,output_widget):
    output_widget.clear_output()

    #create a map zoomed in around Smolensk
    battle_map = folium.Map([54.78, 32.04],zoom_start=6)
    #add a heat map with a time dimension
    HeatMapWithTime(heat_data,auto_play=False,max_opacity=0.7,radius=req_radius).add_to(battle_map)

    with output_widget:
        iframe = battle_map._repr_html_()
        display(HTML(iframe))

#initial call to display default map
display_map(25,out)

#output UI
res = widgets.VBox([radius_sel,out])
res